In [7]:
import logging
# Some packages used by pvtrace are a little noisy
logging.getLogger('trimesh').disabled = True
logging.getLogger('shapely.geos').disabled = True
import numpy as np
from pvtrace import *
#from tutorial_support import interact_ray

In [8]:
def interact_ray(scene, vis):

    ray_ids = []

    def move_ray(x=0.0, y=0.0, z=0.0, theta=0.0, phi=0.0, nanometers=555.0):
        # Clear old objects
        [vis.remove_object(_id) for _id in ray_ids]

        # Create new array with position from the interact UI
        ray = Ray(
            position=(x, y, z),
            direction=(
                np.sin(np.radians(theta)) * np.cos(np.radians(phi)),
                np.sin(np.radians(theta)) * np.sin(np.radians(phi)),
                np.cos(np.radians(theta))
            ),
            wavelength=nanometers
        )

        # Re-create the scene with the new ray but reuse the renderers
        steps = photon_tracer.follow(scene, ray, maxsteps=10)
        path, events = zip(*steps)
        vis.render(scene)

        # Remove old rays; add new rays
        ray_ids.clear()
        ray_ids.extend(vis.add_ray_path(path))

    return interact(
        move_ray,
        x=(-0.6, 0.6, 0.01),
        y=(-0.6, 0.6, 0.01),
        z=(-0.6, 0.6, 0.01),
        theta=(0, 180, 1),
        phi=(0, 360, 1),
        nanometers=(300, 800, 1)
    )

# Interactive ray tracing example

Let's make a scene and use ipython widgets to move the starting point of rays.

All scenes must have a **world node** which contains all other objects.

In [9]:
world = Node(
    name="world (air)",
    geometry=Sphere(
        radius=10.0,
        material=Material(refractive_index=1.0),
    )
)
sphere = Node(
    name="sphere (glass)",
    geometry=Sphere(
        radius=1.0,
        material=Material(refractive_index=1.5),
    ),
    parent=world
)
scene = Scene(world)

The scene is a large world sphere containing air with a smaller glass sphere inside and at the centre.

---

**Units**

*pvtrace* does *not* assume units, they could be metres, kilometres or lightyears.

---

Use the MeshCat renderer to view the scene directly in the Jupyter notebook. The simplest scene just contains a Ray.

In [10]:
renderer = MeshcatRenderer(wireframe=True)
renderer.render(scene)
renderer.vis.jupyter_cell()

KeyboardInterrupt: 

In [ ]:
_ = interact_ray(scene, renderer)

The `interact_ray` function makes a Ray,

In [10]:
ray = Ray(
    position=(0.0, 0.0, 0.0),
    direction=(0.0, 0.0, 1.0),
    wavelength=555.0
)

and every time the one of the slides changes and retraces the scene.

Rays are a simple data structure containing `position`, `direction` and `wavelength`.

## Getting trace data

Use the `follow` function, from the `photon_tracer` module, to get a list of interaction points and event types that a ray made with the scene.

In [6]:
np.random.seed(0)
steps = photon_tracer.follow(scene, ray)

In [7]:
r, e = steps[0]
f"{e.name}: {r}"

'GENERATE: Ray(pos=(0.00, 0.00, 0.00), dir=(0.00, 0.00, 1.00), nm=555.00, alive=True)'

At the first step the ray is generated with the values we specify.

In [8]:
r, e = steps[1]
f"{e.name}: {r}"

'TRANSMIT: Ray(pos=(0.00, 0.00, 1.00), dir=(0.00, 0.00, 1.00), nm=555.00, alive=True)'

The ray hits the small glass sphere and is transmitted. 

In [9]:
r, e = steps[2]
f"{e.name}: {r}"

'EXIT: Ray(pos=(0.00, 0.00, 10.00), dir=(0.00, 0.00, 1.00), nm=555.00, alive=True)'

Finally the ray exits the scene by hitting the world sphere.

Full list of events is described by the enum,

```python
class Event(Enum):
    GENERATE = 0
    REFLECT = 1
    TRANSMIT = 2
    ABSORB = 3
    SCATTER = 4
    EMIT = 5
    EXIT = 6
    KILL = 7
```

This gives a hint of the other capabilities of pvtrace which will be covered in the next tutorials.